# 비지도 학습(unsupervised learning) 기반 형태소 분석


> 데이터의 패턴을 모델 스스로 학습 

> 자주 등장하는 단어들을 형태소로 인식





### ✾ soynlp : 데이터 통계량 기반 단어 점수표로 형태소 분석
> ✎ 단어 점수 : 지도 학습과 달리 별도의 학습 과정을 거쳐서 만들어야 함.
1. 응집 확률(Cohesion Probability) : 함께 자주 나타나는 경우
2. Branching Entropy : 단어 앞뒤로 다양한 조사 or 어미 or 다른 단어가 등장하는 경우. 즉, 앞뒤로 패턴이 다양한 경우

> -> 응집 확률, Branching Entropy 둘 다 높을 때 형태소로 취급.






In [7]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 4.2 MB/s 


In [8]:
from soynlp.word import WordExtractor

corpus_fname = './processed_ratings.txt' # 네이버 영화 리뷰
model_fname = './soyword.model' # 단어 점수표

sentences = [sent.strip() for sent in open(corpus_fname, 'r').readlines()]
word_extractor = WordExtractor(min_frequency=100,
                               min_cohesion_forward=0.05,
                               min_right_branching_entropy=0.0)
word_extractor.train(sentences)                     
word_extractor.save(model_fname)

training was done. used memory 1.042 Gb


In [10]:
import math
from soynlp.tokenizer import LTokenizer

model_fname = './soyword.model'
word_extractor.load(model_fname)

scores = word_extractor.word_scores()
scores = {key:(scores[key].cohesion_forward * math.exp(scores[key].right_branching_entropy)) for key in scores.keys()}
tokenizer = LTokenizer(scores=scores)
tokens = tokenizer.tokenize('애비는 종이었다')
print(tokens)

all cohesion probabilities was computed. # words = 6130
all branching entropies was computed # words = 123575
all accessor variety was computed # words = 123575
['애비는', '종이었다']


In [12]:
# 띄어쓰기 교정 모듈 설치
!pip install soyspacing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
# 말뭉치에서 띄어쓰기 패턴을 학습
from soyspacing.countbase import CountSpace

model_fname = './space-correct.model'
model = CountSpace()
model.train(corpus_fname)
model.save_model(model_fname, json_format=False)

all tags length = 1166880 --> 165338, (num_doc = 199991)

In [15]:
# 띄어쓰기 교정
model.load_model(model_fname, json_format=False)
model.correct('어릴때보고 지금다시봐도 재밌어요')

('어릴때 보고 지금 다시봐도 재밌어요', [0, 0, 1, 0, 1, 0, 1, 0, None, 0, 1, 0, 0, 0, 1])

### ✾ sentencepiece : BPE 활용/구글
> ✎ BPE(Byte Pair Encoding) : 반복적으로 고빈도 문자열들을 병합해 어휘 집합에 추가. 
- 어휘 집합(vocabulary)에 있으면 subword, 없으면 미등록 단어(unknown word)로 취급.


In [ ]:
!pip install sentencepiece